In [4]:
import hl7
from datetime import datetime
from app.models import ErrorCharges,Patient
from asgiref.sync import sync_to_async

file_paths = ['app/HL7/charges_0.hl7', 'app/HL7/charges_1.hl7', 'app/HL7/charges_2.hl7', 'app/HL7/charges_3.hl7', 'app/HL7/charges_4.hl7']
def parse_hl7(file_path):
    with open(file_path, 'br') as file:
        data = file.read()
    return hl7.parse(data)

parsed_messages = [parse_hl7(file_path) for file_path in file_paths]

patient_data = []
def extract_patient_info(message):
    segments = [segment for segment in message]
    for segment1 in segments:
        patient_info = {}
        segment = [segment2 for segment2 in segment1]
        if segment[0] == ['\nPID']:
            fullnamelist = str(segment[5][0]).split('^^')
            namelist = fullnamelist[0].split('^')
            patient_info['full_name'] = fullnamelist[0].replace('^',' ')  if fullnamelist[0].replace('^',' ') != '' else None
            patient_info['first_name'] = namelist[0] if namelist[0] != '' else None
            patient_info['last_name'] = ' '.join(namelist[1:]) if ' '.join(namelist[1:]) != '' else None
            patient_info['gender'] = 'Male' if segment[8][0] == 'M' else 'Female' if segment[8][0] == 'F' else None
            patient_info['phone'] = segment[13][0]
            patient_info['address'] = str(segment[11][0]).replace('^^','-').replace('^',', ')
            patient_info['date_of_birth'] = segment[7][0][0:4]+'-'+segment[7][0][4:6]+'-'+segment[7][0][6:8]  if segment[7][0] != '' else None
            patient_info['mrn'] = list(segment[3][0])[0][0] if list(segment[3][0])[0] != [''] else None
            patient_data.append(patient_info)
    return patient_info


for message in parsed_messages:
    extract_patient_info(message)
    
async def store_patient_data_async(patient_data):
    listvalue = await sync_to_async(list)(Patient.objects.all())
    if len(listvalue) == 0:
        for data in patient_data:
            error_charges_id = None
            if data['full_name'] is None:
                error_charges_id = 1
            if data['date_of_birth'] is None:
                error_charges_id = 2
            if data['mrn'] is None:
                error_charges_id = 3
            if error_charges_id:
                data["error_charges_id"] = error_charges_id
            await sync_to_async(Patient.objects.create)(**data)

await store_patient_data_async(patient_data)